# Content

1. [Import and Data Cleaning](#import) 
2. [Preprocessing](#preprocessing)
 1. [Sentences Preprocessing](#sentences-preprocessing)
 2. [Stopword removal](#stopword-removal)
 3. [Tokenization](#token)

<a id='import'></a><h1>Importing and data cleaning</h1>

In [1]:
!pip install spacy

In [1]:
import pandas as pd
import numpy as np
import json
import string
import re

import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import spacy
from spacy.tokenizer import Tokenizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\benja\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\benja\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# pd.set_option("display.max_rows", None, "display.max_columns", None)

In [2]:
movie_df = pd.read_json('./IMDB_movie_details.json', lines=True)
reviews_df = pd.read_json('./IMDB_reviews.json', lines=True)

In [5]:
# movie_df.to_csv('movies.csv', index=False)
# reviews_df.to_csv('reviews.csv', index=False)

In [3]:
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1572 entries, 0 to 1571
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   movie_id       1572 non-null   object 
 1   plot_summary   1572 non-null   object 
 2   duration       1572 non-null   object 
 3   genre          1572 non-null   object 
 4   rating         1572 non-null   float64
 5   release_date   1572 non-null   object 
 6   plot_synopsis  1572 non-null   object 
dtypes: float64(1), object(6)
memory usage: 86.1+ KB


In [7]:
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 573913 entries, 0 to 573912
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   review_date     573913 non-null  object
 1   movie_id        573913 non-null  object
 2   user_id         573913 non-null  object
 3   is_spoiler      573913 non-null  bool  
 4   review_text     573913 non-null  object
 5   rating          573913 non-null  int64 
 6   review_summary  573913 non-null  object
dtypes: bool(1), int64(1), object(5)
memory usage: 26.8+ MB


In [8]:
# movie_df.head()
# reviews_df.head()

Remove movie ids with '/' at the end. Check if ids in movie_df are in reviews_df and vice versa. No need to drop any rows as all ids are present in both datasets

In [9]:
id_cleaning = ['tt0104014/', 'tt0114142/']

for ele in id_cleaning:
    idx = movie_df[movie_df['movie_id'] == ele].index
    movie_df.loc[idx, ['movie_id']] = ele[:-1]

movie_unique_id = list(pd.unique(movie_df['movie_id']))
reviews_unique_id = pd.unique(reviews_df['movie_id'])

not_found_movie = []
not_found_reviews = []

for ele in movie_unique_id: 
    if ele not in reviews_unique_id: 
        not_found_movie.append(ele)
        
for ele in reviews_unique_id: 
    if ele not in movie_unique_id: 
        not_found_reviews.append(ele)
        
print(not_found_reviews)
print(not_found_movie)

[]
[]


Drop rows with empty synopsis or reviews. Empty columns do not come up as null, hence this step.

In [10]:
dropped_index = list(movie_df[movie_df['plot_synopsis'].values == ''].index)

In [11]:
dropped_ids = []

for idx in dropped_index: 
    dropped_ids.append(movie_df.iloc[idx]['movie_id'])
    
reviews_df = reviews_df[~reviews_df['movie_id'].isin(dropped_ids)]

In [12]:
movie_df.drop(dropped_index, inplace=True)

In [13]:
reviews_df = reviews_df.drop(reviews_df[reviews_df['review_summary'].values == ''].index)

In [14]:
movie_df = movie_df.reset_index(drop=True)
reviews_df = reviews_df.reset_index(drop=True)

In [9]:
movie_df

,movie_id,plot_summary,duration,genre,rating,release_date,plot_synopsis
0,tt0105112,"Former CIA analyst, Jack Ryan is in England wi...",1h 57min,"[Action, Thriller]",6.9,1992-06-05,"Jack Ryan (Ford) is on a ""working vacation"" in..."
1,tt1204975,"Billy (Michael Douglas), Paddy (Robert De Niro...",1h 45min,[Comedy],6.6,2013-11-01,Four boys around the age of 10 are friends in ...
2,tt0040897,"Fred C. Dobbs and Bob Curtin, both down on the...",2h 6min,"[Adventure, Drama, Western]",8.3,1948-01-24,Fred Dobbs (Humphrey Bogart) and Bob Curtin (T...
3,tt0126886,Tracy Flick is running unopposed for this year...,1h 43min,"[Comedy, Drama, Romance]",7.3,1999-05-07,Jim McAllister (Matthew Broderick) is a much-a...
4,tt0286716,"Bruce Banner, a brilliant scientist with a clo...",2h 18min,"[Action, Sci-Fi]",5.7,2003-06-20,Bruce Banner (Eric Bana) is a research scienti...
...,...,...,...,...,...,...,...
1334,tt0120655,An abortion clinic worker with a special herit...,2h 10min,"[Adventure, Comedy, Drama]",7.3,1999-11-12,The film opens with a homeless man (Bud Cort) ...
1335,tt0276751,Twelve year old Marcus Brewer lives with his c...,1h 41min,"[Comedy, Drama, Romance]",7.1,2002-05-17,Will Freeman (Hugh Grant) is a 38-year-old bac...
1336,tt0289879,Evan Treborn grows up in a small town with his...,1h 53min,"[Sci-Fi, Thriller]",7.7,2004-01-23,"In the year 1998, Evan Treborn (Ashton Kutcher..."
1337,tt1723811,Brandon is a 30-something man living in New Yo...,1h 41min,[Drama],7.2,2012-01-13,"Brandon (Michael Fassbender) is a successful, ..."


In [10]:
reviews_df

,review_date,movie_id,user_id,is_spoiler,review_text,rating,review_summary
0,10 February 2006,tt0111161,ur1898687,True,"In its Oscar year, Shawshank Redemption (writt...",10,A classic piece of unforgettable film-making.
1,6 September 2000,tt0111161,ur0842118,True,The Shawshank Redemption is without a doubt on...,10,Simply amazing. The best film of the 90's.
2,3 August 2001,tt0111161,ur1285640,True,I believe that this film is the best story eve...,8,The best story ever told on film
3,1 September 2002,tt0111161,ur1003471,True,"**Yes, there are SPOILERS here**This film has ...",10,Busy dying or busy living?
4,20 May 2004,tt0111161,ur0226855,True,At the heart of this extraordinary movie is a ...,8,"Great story, wondrously told and acted"
...,...,...,...,...,...,...,...
538821,8 August 1999,tt0139239,ur0100166,False,"Go is wise, fast and pure entertainment. Assem...",10,The best teen movie of the nineties
538822,31 July 1999,tt0139239,ur0021767,False,"Well, what shall I say. this one´s fun at any ...",9,Go - see the movie
538823,20 July 1999,tt0139239,ur0392750,False,"Go is the best movie I have ever seen, and I'v...",10,It's the best movie I've ever seen
538824,11 June 1999,tt0139239,ur0349105,False,Call this 1999 teenage version of Pulp Fiction...,3,Haven't we seen this before?


<a id='preprocessing'></a><h1>Preprocessing</h1>

Convert boolean values into int

In [15]:
reviews_df['is_spoiler'] = reviews_df['is_spoiler'].replace({True:1, False:0})

In [12]:
reviews_df.head(100)

,review_date,movie_id,user_id,is_spoiler,review_text,rating,review_summary
0,10 February 2006,tt0111161,ur1898687,1,"In its Oscar year, Shawshank Redemption (writt...",10,A classic piece of unforgettable film-making.
1,6 September 2000,tt0111161,ur0842118,1,The Shawshank Redemption is without a doubt on...,10,Simply amazing. The best film of the 90's.
2,3 August 2001,tt0111161,ur1285640,1,I believe that this film is the best story eve...,8,The best story ever told on film
3,1 September 2002,tt0111161,ur1003471,1,"**Yes, there are SPOILERS here**This film has ...",10,Busy dying or busy living?
4,20 May 2004,tt0111161,ur0226855,1,At the heart of this extraordinary movie is a ...,8,"Great story, wondrously told and acted"
...,...,...,...,...,...,...,...
95,13 September 2015,tt0111161,ur1951246,1,Two imprisoned men bond over a number of years...,10,a spotted sacrifice through darkness leads to ...
96,28 June 2015,tt0111161,ur61209717,1,"In its Oscar year, Shawshank Redemption (writt...",8,Review
97,17 March 2015,tt0111161,ur56613094,1,I have seen the movie for the first time. It's...,8,Beautiful
98,1 March 2015,tt0111161,ur58986962,1,One of the classic movie ever made. Lots of en...,10,Classic Movie


## Cleaning synopsis and summary

1. Expand Contractions - pip install contractions/use json
2. Lower case 
3. Remove punctuations - use string 
4. Remove words and digits containing digits
5. Remove Stopwords - use nltk
6. Stemming and Lemmatization - use nltk
7. Remove extra spaces

<a id='sentences-preprocessing'></a><h2>Sentences preprocessing</h2>

Steps done: 
1. Expand Contractions - use json
2. Lower case 
3. Remove punctuations
4. Remove words and digits containing digits

In [16]:
with open ('./contractions.json') as f:
    contractions_dict = json.load(f)

def contractions_cleaning(sentence):
    return ' '.join([contractions_dict.get(word, word) for word in sentence.split()])

In [17]:
def sentence_preprocessing(df, col):
    df[col] = df[col].str.lower() # convert to lowercase
    df[col] = df[col].apply(contractions_cleaning) # expand contractions
    df[col] = df[col].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ' , x)) # remove puncutations
    df[col] = df[col].apply(lambda x : re.sub(r"\d+", " ", x)) # remove numbers
    return df[col]

reviews_df['review_text'] = sentence_preprocessing(reviews_df, 'review_text')
reviews_df['review_summary'] = sentence_preprocessing(reviews_df, 'review_summary')
movie_df['plot_summary'] = sentence_preprocessing(movie_df, 'plot_summary')
movie_df['plot_synopsis'] = sentence_preprocessing(movie_df, 'plot_synopsis')

In [17]:
print(movie_df['plot_synopsis'].loc[40])

story of gerry conlon  purported ringleader of the guildford four  a group of three irishmen and one english woman wrongly imprisoned for the   ira bombing of a pub in guildford  england  that left five people dead  conlon s father guiseppe was subsequently imprisoned along with six other conlon relatives who became known as the maguire seven                                      the story opens with a flash forward to the bombing of a pub in the guildford suburb of london  the pub  frequented by british soldiers  was targeted by the irish republican army  ira  as an act of terrorism a woman driving a car listens to a cassette recording of gerry conlon  a northern irish political prisoner  gerry speaks of his younger years living in belfast where he had steal lead sheets from the roofs of row houses and sell them  one day  while holding a piece of piping  he is shot at by british soldiers who mistake him for a sniper  believing the pipe is a rifle  gerry runs into the streets and throug

</a><a id='stopword-removal'></a><h2>Stopword Removal</h2>

In [18]:
stop_words = set(stopwords.words('english'))

def remove_stopwords(sentence):
  return ' '.join([word for word in sentence.split() if not word in stop_words])

reviews_df['review_text'] = reviews_df['review_text'].apply(remove_stopwords)
reviews_df['review_summary'] = reviews_df['review_summary'].apply(remove_stopwords)
movie_df['plot_summary'] = movie_df['plot_summary'].apply(remove_stopwords)
movie_df['plot_synopsis'] = movie_df['plot_synopsis'].apply(remove_stopwords)


print(movie_df['plot_synopsis'].loc[40])

story gerry conlon purported ringleader guildford four group three irishmen one english woman wrongly imprisoned ira bombing pub guildford england left five people dead conlon father guiseppe subsequently imprisoned along six conlon relatives became known maguire seven story opens flash forward bombing pub guildford suburb london pub frequented british soldiers targeted irish republican army ira act terrorism woman driving car listens cassette recording gerry conlon northern irish political prisoner gerry speaks younger years living belfast steal lead sheets roofs row houses sell one day holding piece piping shot british soldiers mistake sniper believing pipe rifle gerry runs streets several homes chased inadvertently starts small riot joins mob throwing rocks molotov cocktails members ira using one houses gerry runs munitions cache find gerry threaten shoot leg gerry father giuseppe told daughters brother trouble rushes scene talks ira leader tells giuseppe trying scare gerry behaving

In [19]:
import matplotlib.pyplot as plt
import numpy as np
final_df = pd.merge(movie_df,reviews_df, on='movie_id', how = 'inner')

In [20]:
final_df2 = final_df.drop(['duration','genre','rating_x','release_date','user_id','rating_y','review_date','movie_id'],axis=1)
final_df2.head(3)


,plot_summary,plot_synopsis,is_spoiler,review_text,review_summary
0,former cia analyst jack ryan england family va...,jack ryan ford working vacation london family ...,1,second tom clancy novel made film hunt red oct...,decent clancy thriller
1,former cia analyst jack ryan england family va...,jack ryan ford working vacation london family ...,1,second looks like becoming jack ryan catalogue...,jack ryan chapter ii
2,former cia analyst jack ryan england family va...,jack ryan ford working vacation london family ...,1,fan hunt red october boring opinion alec baldw...,filled brim excitement beats red october count...


In [21]:
last = final_df2.groupby('is_spoiler').apply(lambda x: x.sample(1000))
last.shape
last['is_spoiler'].value_counts()


0    1000
1    1000
Name: is_spoiler, dtype: int64

In [44]:
# def count_len(text):
#   wordlist=text.split(' ')
#   return len(wordlist)

# last['len_lemmatized']=last['review_text'].apply(count_len)
# print(last['len_lemmatized'].mean())

In [34]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Embedding, Dense, LSTM, SpatialDropout1D
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
texts = last['plot_summary'].values
tokenizer2 = Tokenizer()
tokenizer2.fit_on_texts(texts)
X = tokenizer2.texts_to_sequences(texts)
#X_lemmatized = Tokenizer.texts_to_sequences(df_to_use['plot_summary'].values)

#X_train for plot_summary
X_train = pad_sequences(X, maxlen= 200)#MAX_SEQUENCE_LENGTH)
y_train = pd.get_dummies(last['is_spoiler']).values

texts_synopsis = last['plot_synopsis'].values
tokenizer2 = Tokenizer()
tokenizer2.fit_on_texts(texts_synopsis)
X1 = tokenizer2.texts_to_sequences(texts_synopsis)
X_train1 = pad_sequences(X1, maxlen=200)

texts_review = last['review_text'].values
tokenizer2 = Tokenizer()
tokenizer2.fit_on_texts(texts_review)
X2 = tokenizer2.texts_to_sequences(texts_review)
X_train2 = pad_sequences(X2, maxlen=200)


print(len(y_train))

2000


In [35]:
#validation set

val = final_df2.groupby('is_spoiler').apply(lambda x: x.sample(1000))
texts_val = val['plot_summary'].values
tokenizer2 = Tokenizer()
tokenizer2.fit_on_texts(texts_val)
X_v = tokenizer2.texts_to_sequences(texts_val)
X_val = pad_sequences(X_v, maxlen= 200)
y_test= pd.get_dummies(val['is_spoiler']).values

val_synopsis = val['plot_synopsis'].values
tokenizer2 = Tokenizer()
tokenizer2.fit_on_texts(val_synopsis)
X_v1 = tokenizer2.texts_to_sequences(val_synopsis)
X_val1 = pad_sequences(X_v1, maxlen=200)

val_review = val['review_text'].values
tokenizer2 = Tokenizer()
tokenizer2.fit_on_texts(val_review)
X_v2 = tokenizer2.texts_to_sequences(val_review)
X_val2 = pad_sequences(X_v2, maxlen=200)
print(len(y_test))

2000


In [36]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 20000

# Most reviews have below 200 words. Standardize input fed into the model, every review will cut off at 200 words. Pad reviews with less than 200 words.
MAX_SEQUENCE_LENGTH = 200
EMBEDDING_DIM = 100

In [41]:
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import concatenate, Dropout

lemmatized_input = Input(shape=(200,))
lemmatized_input1 = Input(shape=(200,))
lemmatized_input2 = Input(shape=(200,))

#operate on lemmatized inputs
x = Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=200)(lemmatized_input)
x = SpatialDropout1D(0.2)(x)
x = LSTM(64, dropout=0.2)(x)
x = Dense(16, activation="relu")(x)
x = Dense(4, activation="relu")(x)
x = Model(inputs=lemmatized_input, outputs=x)

y = Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=200)(lemmatized_input1)
y = SpatialDropout1D(0.2)(y)
y = LSTM(64, dropout=0.2)(y)
y = Dense(16, activation="relu")(y)
y = Dense(4, activation="relu")(y)
y = Model(inputs=lemmatized_input1, outputs=y)

v = Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=200)(lemmatized_input2)
v = SpatialDropout1D(0.2)(v)
v = LSTM(64, dropout=0.2)(v)
v = Dense(16, activation="relu")(v)
v = Dense(4, activation="relu")(v)
v = Model(inputs=lemmatized_input2, outputs=v)


#operate on remaining inputs
# y = Dense(8, activation="relu")(remaining_inputs)
# y = Dense(4, activation="relu")(y)
# y = Model(inputs=remaining_inputs, outputs=y)

# combined = concatenate([x.output, y.output])

z = Dense(4, activation="relu")(x.output)
z = Dense(2, activation="sigmoid")(z)


model = Model(inputs=[x.input, y.input, v.input], outputs=z)

In [42]:
model.compile(optimizer=tf.optimizers.Adam(), 
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [43]:
model.fit([X_train, X_train1, X_train2], y_train, epochs=12, batch_size=64, validation_data=([X_val, X_val1, X_val2], y_test), callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001, restore_best_weights=True)])
# model.fit(np.array(X_train), np.array(y_train), epochs=12, batch_size=64, validation_data=(np.array(X_val), np.array(y_test)), callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001, restore_best_weights=True)])

Epoch 1/12
32/32 [==============================] - 8s 183ms/step - loss: 0.6932 - accuracy: 0.4735 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 2/12
32/32 [==============================] - 5s 171ms/step - loss: 0.6931 - accuracy: 0.5085 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 3/12
32/32 [==============================] - 5s 170ms/step - loss: 0.6917 - accuracy: 0.5870 - val_loss: 0.6931 - val_accuracy: 0.5265
Epoch 4/12
32/32 [==============================] - 5s 171ms/step - loss: 0.6867 - accuracy: 0.7045 - val_loss: 0.6936 - val_accuracy: 0.5290


In [22]:
# SAVE TO CSV
# import os
# final_df = pd.merge(movie_df,reviews_df, on='movie_id', how = 'inner')
# final_df.head(3)
# final_df.drop(['user_id'],axis=1)
# final_df.head(3)
# final_df = final_df.drop(['duration','genre','rating_x','release_date','user_id','rating_y','review_date','movie_id'],axis=1)
# final_df.head(3)
# final_df.to_csv(os.getcwd() + 'merged1.csv', index=False)


</a><a id='token'></a><h2>Tokenization</h2>

In [16]:
import spacy
from spacy.tokenizer import Tokenizer

In [24]:
# if it dont work, open anaconda prompt and install run this
# python -m spacy download en_core_web_lg

In [17]:
nlp = spacy.load('en_core_web_lg')
tokenizer = Tokenizer(nlp.vocab)

reviews_df['review_text'] = reviews_df['review_text'].apply(lambda x: [ token.text for token in tokenizer(x)])
reviews_df['review_summary'] = reviews_df['review_summary'].apply(lambda x: [ token.text for token in tokenizer(x)])
movie_df['token_summary'] = movie_df['plot_summary'].apply(lambda x: [ token.text for token in tokenizer(x)])
movie_df['token_synopsis'] = movie_df['plot_synopsis'].apply(lambda x: [ token.text for token in tokenizer(x)])

In [38]:
!pip install tensorflow

  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4848 sha256=9ebe3364e49ce962ad90404a70acd706d5e8c87dad92f15f154f3cd54c066f3c
  Stored in directory: c:\users\benja\appdata\local\pip\cache\wheels\3f\e3\ec\8a8336ff196023622fbcb36de0c5a5c218cbb24111d1d4c7f2
Successfully built termcolor
